# SETUP

In [1]:
import pandas as pd 
import pyreadr as py 
import numpy as np 
import tensorflow as tf
import gc 
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn import preprocessing
import matplotlib as plt
tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions


gc.collect() 

pd.options.mode.chained_assignment = None

In [2]:
# #TPU

# import os
# assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
# TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# print('TPU address is', TPU_ADDRESS)

# # Detect hardware, return appropriate distribution strategy
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)

TPU address is grpc://10.87.159.114:8470
Running on TPU  grpc://10.87.159.114:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.87.159.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.87.159.114:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# HYPER

In [51]:
BASE_SEQ_LEN = 4467
NUM_VAR = 31
INPUT_SEQ_LEN = 16 
OUTPUT_SEQ_LEN = 1 
BATCH_SIZE = 128

# READ DATA

In [18]:
url = 'https://raw.githubusercontent.com/Rainmaker519/NGAFID_Anomaly_Detection/main/NGAFID_dataset/c172_file_2.csv'
data = pd.read_csv(url)

In [19]:
data.describe()

,AltAGL,AltB,AltGPS,AltMSL,BaroA,E1_CHT1,E1_CHT2,E1_CHT3,E1_CHT4,E1_EGT1,E1_EGT2,E1_EGT3,E1_EGT4,E1_FFlow,E1_OilP,E1_OilT,E1_RPM,FQtyL,FQtyR,GndSpd,IAS,LatAc,NormAc,OAT,Pitch,Roll,TAS,VSpd,VSpdG,WndDr,WndSpd
count,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000,4467.000000
mean,621.904634,1447.365413,1354.174905,1443.162077,30.003956,282.680786,306.380663,309.696344,322.298849,1271.838117,1240.913891,1269.034569,1242.800405,8.043550,63.439507,166.060369,1923.664607,13.878471,16.101974,78.021084,76.867974,-0.012691,0.024941,21.816835,2.927833,-2.875534,79.852026,-15.005946,-15.201948,61.420081,5.150486
std,430.763101,419.003121,422.684410,422.680796,0.009185,23.679792,26.798223,24.313926,26.057577,113.993135,151.593224,114.742380,110.930938,4.633955,7.356695,5.454409,597.475491,2.211355,1.415391,26.455299,28.560461,0.029323,0.103672,1.108857,3.980810,12.151214,29.889523,488.473980,517.127489,103.837486,3.116676
min,0.000000,825.500000,739.300000,828.500000,29.990000,197.340000,201.590000,219.090000,225.070000,990.490000,797.260000,993.090000,957.750000,1.710000,42.950000,150.220000,767.800000,8.330000,9.790000,0.000000,-1.000000,-0.420000,-1.120000,19.200000,-22.420000,-58.460000,-1.000000,-3850.600000,-4169.300000,-179.900000,0.000000
25%,218.000000,1057.900000,960.900000,1050.150000,29.990000,267.375000,290.080000,292.560000,304.485000,1176.890000,1123.615000,1181.035000,1155.965000,2.475000,55.340000,163.085000,1237.900000,11.960000,15.230000,70.080000,69.390000,-0.030000,-0.020000,21.500000,0.525000,-3.440000,72.000000,-318.070000,-330.700000,0.000000,2.530000
50%,705.000000,1529.900000,1432.900000,1522.100000,30.010000,289.760000,314.860000,317.460000,330.750000,1322.650000,1327.840000,1322.010000,1302.580000,8.780000,67.290000,167.760000,2225.200000,13.540000,16.100000,81.390000,79.290000,-0.010000,0.010000,21.800000,2.320000,-0.100000,82.000000,-9.520000,-3.900000,101.900000,5.250000
75%,814.500000,1642.400000,1540.850000,1629.850000,30.010000,299.965000,322.860000,327.235000,341.645000,1346.820000,1347.425000,1353.805000,1322.390000,10.575000,68.660000,169.865000,2406.500000,15.650000,17.150000,97.520000,97.245000,0.000000,0.050000,22.800000,4.905000,1.310000,101.000000,220.415000,216.500000,148.100000,7.730000
max,1365.000000,2203.500000,2089.900000,2178.800000,30.010000,319.640000,351.060000,348.560000,360.260000,1416.460000,1394.960000,1394.470000,1359.690000,15.520000,75.490000,173.800000,2618.100000,21.830000,21.090000,123.300000,126.200000,0.180000,0.850000,24.000000,22.550000,31.890000,132.000000,3669.790000,3858.300000,179.900000,12.560000


# MODEL

In [69]:
def get_slice_random_segment(length):
    """This function will acquire a length of slice from the dataset.
    Parameter: length, length of the dataset
    return: array
    """
    def slice_random_segment(x):
      """This function will return the specific array
      Parameter: x, dataset
      return: array
      """
      #minval = min numbers of data points, maxval = max numbers of data points
      start = tf.random.uniform(shape=[], minval=0, maxval= BASE_SEQ_LEN-length, dtype=tf.int64)
      x = x[start:start+length]
      #reshape will turn x into the shape of (length, NUM_VAR)
      x = tf.reshape(x, (length, NUM_VAR))
      return x 
    return slice_random_segment

def get_split_xy(y_length):
  """This will get two specific group of data points.
  Parameter: y_length
  return: x, y 
  """
  def split_xy(x):
    """
    Parameter: dataset
    """
    y = x[-y_length:]
    return x[:-y_length], y 
  return split_xy

def get_dataset(train_data, shuffle = False, repeat = False): 
    ds = tf.data.Dataset.from_tensor_slices(train_data)
    print(ds)
    ds = ds.map(get_slice_random_segment(length = INPUT_SEQ_LEN + OUTPUT_SEQ_LEN ))
    ds = ds.map(get_split_xy(y_length=OUTPUT_SEQ_LEN))
    print(ds)

    #Changing the value of buffer_size affects how uniform the shuffling
    ds = ds.shuffle(512) if shuffle else ds 
    ds = ds.repeat() if repeat else ds.repeat(2)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)

    return ds 


def get_detection_xy(df):
    """get the testing dataset
    Parameter: dataset
    return: x:slice of dataset(testing), y:target value
    """
    scaled = scaler.transform(df)

    x = []
    y = []

    for i in range(BASE_SEQ_LEN - INPUT_SEQ_LEN - OUTPUT_SEQ_LEN):
        x.append(scaled[i:i+ INPUT_SEQ_LEN])
        y.append(scaled[i+INPUT_SEQ_LEN: i + INPUT_SEQ_LEN + OUTPUT_SEQ_LEN])

    x = np.stack(x)
    y = np.stack(y)

    return x,y 
    
def get_mse(df, model):
    """ Get loss value: the mean squared error value
    Parameter: dataset, model
    Return: loss value
    """
    x, y = get_detection_xy(df) 
    y_pred_1 = model.predict(x, verbose = True)
    mse_1 = tfk.losses.MSE(y, y_pred_1).numpy()
    return mse_1.flatten()


In [53]:
#LSTM
def get_model_1():
    with strategy.scope():
      model = tf.keras.Sequential([
                              tf.keras.Input(shape  = (INPUT_SEQ_LEN, NUM_VAR)),
                              tfkl.LSTM(64, return_sequences=True), 
                              tfkl.LSTM(64, return_sequences=False),
                              tfkl.RepeatVector(OUTPUT_SEQ_LEN), 
                              tfkl.LSTM(64, return_sequences=True),
                              tfkl.Dense(NUM_VAR, activation = 'relu')

      ])

      model.compile(optimizer = tfk.optimizers.Adam(learning_rate=1e-4), loss = "mse" )

      return model 

In [ ]:
#LSTM
def get_model_2():
    with strategy.scope():
      model = tf.keras.Sequential([
                              tf.keras.Input(shape  = (INPUT_SEQ_LEN, NUM_VAR)),
                              tfkl.LSTM(64, return_sequences=True), 
                              tfkl.LSTM(64, return_sequences=True),
                              tfkl.LSTM(64, return_sequences=False),
                              tfkl.Dropout(0.5),
                              tfkl.RepeatVector(OUTPUT_SEQ_LEN), 
                              tfkl.LSTM(64, return_sequences=True),
                              tfkl.Dense(NUM_VAR, activation = 'relu')

      ])

      model.compile(optimizer = tfk.optimizers.Adam(learning_rate=1e-4), loss = "mse" )

      return model 

In [ ]:
#GRU
def get_model_3():
    with strategy.scope():
      model = tf.keras.Sequential([
                              tf.keras.Input(shape  = (INPUT_SEQ_LEN, NUM_VAR)),
                              tfkl.GRU(64, return_sequences=True), 
                              tfkl.GRU(64, return_sequences=False),
                              tfkl.RepeatVector(OUTPUT_SEQ_LEN), 
                              tfkl.GRU(64, return_sequences=True),
                              tfkl.Dense(NUM_VAR, activation = 'relu')

      ])

      model.compile(optimizer = tfk.optimizers.Adam(learning_rate=1e-4), loss = "mse" )

      return model 

In [ ]:
#GRU
def get_model_4():
    with strategy.scope():
      model = tf.keras.Sequential([
                              tf.keras.Input(shape  = (INPUT_SEQ_LEN, NUM_VAR)),
                              # tfkl.Conv1D(64, 5, 1),
                              tfkl.GRU(64, return_sequences=True), 
                              tfkl.GRU(64, return_sequences=True),
                              tfkl.GRU(64, return_sequences=False),
                              tfkl.Dropout(0.5),
                              tfkl.RepeatVector(OUTPUT_SEQ_LEN), 
                              tfkl.GRU(64, return_sequences=True),
                              tfkl.Dense(NUM_VAR, activation = 'relu')

      ])

      model.compile(optimizer = tfk.optimizers.Adam(learning_rate=1e-4), loss = "mse" )

      return model 


In [61]:
def train_model(df, model):
  # train_df = fault_free_training
  train_df = df

  scaler = preprocessing.MinMaxScaler()
  scaler.fit(train_df.iloc[:,:].values)

  
  train_df.iloc[:,:] = scaler.transform(train_df.iloc[:,:].values)
  print(train_df)
  train_ds = get_dataset(train_df, shuffle = True, repeat=True)
  print(train_ds)
  model.fit(
      train_ds, 
      epochs = 20,
      steps_per_epoch = 1000,
  )

  return model

In [70]:
model = get_model_1()
model = train_model(data, model)

        AltAGL      AltB    AltGPS  ...     VSpdG     WndDr    WndSpd
0     0.793407  0.776052  0.803421  ...  0.424224  0.412173  0.158439
1     0.788278  0.768070  0.794906  ...  0.427662  0.404113  0.157643
2     0.779487  0.763716  0.786317  ...  0.430116  0.395775  0.156051
3     0.770696  0.757184  0.777654  ...  0.430602  0.385492  0.154459
4     0.762637  0.748476  0.769140  ...  0.432072  0.372429  0.152070
...        ...       ...       ...  ...       ...       ...       ...
4462  0.000000  0.007983  0.008663  ...  0.519371  0.500000  0.000000
4463  0.000000  0.007983  0.008589  ...  0.519371  0.500000  0.000000
4464  0.000000  0.007983  0.008589  ...  0.519371  0.500000  0.000000
4465  0.000000  0.008708  0.008441  ...  0.518885  0.500000  0.000000
4466  0.000000  0.008708  0.008441  ...  0.519371  0.500000  0.000000

[4467 rows x 31 columns]
<TensorSliceDataset shapes: (31,), types: tf.float64>
<MapDataset shapes: ((16, 31), (1, 31)), types: (tf.float64, tf.float64)>
<Batch

InvalidArgumentError: ignored

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 64)            29952     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 64)             33024     
_________________________________________________________________
dense (Dense)                (None, 1, 52)             3380      
Total params: 99,380
Trainable params: 99,380
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2 = get_model_2()
model_2 = train_model(fault_free_training, model_2) 

Epoch 1/20
1000/1000 [==============================] - 17s 9ms/step - loss: 0.0420
Epoch 2/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0191
Epoch 3/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0167
Epoch 4/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0158
Epoch 5/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0155
Epoch 6/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0154
Epoch 7/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0153
Epoch 8/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0152
Epoch 9/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0151
Epoch 10/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0151
Epoch 11/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0150
Epoch 12/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0150


In [ ]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32, 64)            29952     
_________________________________________________________________
lstm_4 (LSTM)                (None, 32, 64)            33024     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 1, 64)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 64)             33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 52)            

In [ ]:
model_3 = get_model_3()
model_3 = train_model(fault_free_training, model_3)

Epoch 1/20
1000/1000 [==============================] - 14s 8ms/step - loss: 0.0674
Epoch 2/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0539
Epoch 3/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0531
Epoch 4/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0490
Epoch 5/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0421
Epoch 6/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0416
Epoch 7/20
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0413
Epoch 8/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0410
Epoch 9/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0408
Epoch 10/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0406
Epoch 11/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0404
Epoch 12/20
1000/1000 [==============================] - 8s 8ms/step - loss: 0.040

In [ ]:
model_3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 32, 64)            22656     
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24960     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 1, 64)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 1, 64)             24960     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 52)             3380      
Total params: 75,956
Trainable params: 75,956
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_4 = get_model_4()
model_4 = train_model(fault_free_training, model_4)

Epoch 1/20
1000/1000 [==============================] - 17s 9ms/step - loss: 0.0410
Epoch 2/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0221
Epoch 3/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0201
Epoch 4/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0196
Epoch 5/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0193
Epoch 6/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0189
Epoch 7/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0186
Epoch 8/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0150
Epoch 9/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0139
Epoch 10/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0137
Epoch 11/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0135
Epoch 12/20
1000/1000 [==============================] - 9s 9ms/step - loss: 0.0133


In [ ]:
model_4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 32, 64)            22656     
_________________________________________________________________
gru_4 (GRU)                  (None, 32, 64)            24960     
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 1, 64)             0         
_________________________________________________________________
gru_6 (GRU)                  (None, 1, 64)             24960     
_________________________________________________________________
dense_3 (Dense)              (None, 1, 52)            

# Get the result

In [ ]:
def get_result(model, baseline, simrun):
  mse_model_1 = get_mse(fault_free_testing, simrun, model)
  mse_model_2 = get_mse(faulty_testing, simrun, model)
  pd.DataFrame({'normal' : mse_model_1, 'with_fault' : mse_model_2, 'baseline': baseline}).plot(ylim = (0.0, 1.0))

In [ ]:
baseline = 0.1
run = 1
models = [model, model_2, model_3, model_4]


for i in range(len(models)):
  get_result(models[i], 0.1, run)

NameError: ignored

In [ ]:
#cross validation
for train_split, val_split in KFold().split([i+1 for i in range(500)]):
    train_df = fault_free_training[fault_free_training.simulationRun.isin(train_split)]
    val_df = fault_free_training[fault_free_training.simulationRun.isin(val_split)]


    scaler = preprocessing.MinMaxScaler()
    scaler.fit(train_df.iloc[:, 3:].values)

    train_df.iloc[:, 3:] = scaler.transform(train_df.iloc[:, 3:].values)
    val_df.iloc[:, 3:] = scaler.transform(val_df.iloc[:, 3:].values)

    train_ds = get_dataset(train_df, shuffle = True, repeat=True)
    val_ds = get_dataset(val_df)


    model = get_model() 


    model.fit(
        train_ds, 
        epochs = 20,
        steps_per_epoch = 1000,
        validation_data= val_ds,
        
    )

    break


Epoch 1/20
1000/1000 [==============================] - 17s 11ms/step - loss: 0.1071 - val_loss: 0.0980
Epoch 2/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0744 - val_loss: 0.0678
Epoch 3/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0604 - val_loss: 0.0538
Epoch 4/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0501 - val_loss: 0.0491
Epoch 5/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0485 - val_loss: 0.0479
Epoch 6/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0484 - val_loss: 0.0485
Epoch 7/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0451 - val_loss: 0.0421
Epoch 8/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0424 - val_loss: 0.0407
Epoch 9/20
1000/1000 [==============================] - 10s 10ms/step - loss: 0.0420 - val_loss: 0.0427
Epoch 10/20
1000/1000 [==============================] - 10s 10m

In [ ]:
fault_free_training.shape

(250000, 55)

In [ ]:
#test  cnn
#CNN
def get_cnn_1():
    with strategy.scope():
      model = tf.keras.Sequential([
                              tf.keras.Input(shape  = (INPUT_SEQ_LEN, NUM_VAR)),
                              tfkl.Conv1D(128, 3, strides = 1, padding = 'same', activation= 'relu'), 
                              # tfkl.Conv1D(64, return_sequences=False),
                              # tfkl.RepeatVector(OUTPUT_SEQ_LEN), 
                              # tfkl.Conv1D(64, return_sequences=True),
                              tf.keras.layers.GlobalAveragePooling1D(),
                              tfkl.Dense(NUM_VAR, activation='sigmoid')

      ])
      model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

      return model 